In [13]:
from model.dnn import Genomic_Feature_Extractor  
import torch
config_dims = {'genomic_dim': 20, 'genomic_embedding_dim': 8}
model = Genomic_Feature_Extractor(genomic_dim=config_dims['genomic_dim'],
            genomic_embedding_dim=config_dims['genomic_embedding_dim'])
weights = torch.load('/home/zow/Multi-Cancer/Checkpoints/Trainer/tcga_multi_dnn_trainer_cross_validation_4_bootstrap_test/20230927152936/checkpoint-epoch50.pth')


In [17]:

# get all the genomic feature extractor weights
feat_extractor = weights['models']['Feature_Extractor']
# get the first layer weights, feat_extractor is an ordered dict
# first_key, first_value = next(iter(feat_extractor.items()))
# first_key, first_value

# transform ordered dict to a normal dict
feat_extractor = dict(feat_extractor)
# get the first layer weights
first_layer_weights = feat_extractor['genomic_feature_extractor.genomic_feature_extractor.0.weight']
first_layer_weights


tensor([[-0.4250,  0.2636, -0.4603,  0.0733,  0.4305,  0.0300,  0.2570, -0.4492,
         -0.1545,  0.1972,  0.1246,  0.0849, -0.1533, -0.1209,  0.2893,  0.5609,
         -0.3924,  0.1108, -0.2439,  0.0524],
        [ 0.3815,  0.0620, -0.3089,  0.1661,  0.0566,  0.2817, -0.0772, -0.0423,
         -0.0132,  0.2168,  0.0512, -0.0400,  0.0677, -0.1994,  0.0775,  0.3982,
         -0.0377,  0.1775, -0.5042, -0.3299],
        [-0.5778, -0.1753, -0.0905,  0.1218,  0.1970, -0.3466, -0.4813,  0.1017,
          0.8698, -0.2419,  0.1321, -0.1542,  0.0931, -0.3106,  0.6785, -0.3515,
          0.4746, -0.3496,  0.2080, -0.2929],
        [ 0.5740,  0.0574,  0.3716,  0.1401, -0.2224, -0.3132,  0.1873,  0.6987,
          0.1025, -0.4229,  0.1138,  0.1346,  0.0536, -0.1835, -0.1689, -0.2796,
         -0.1292, -0.0406, -0.0358,  0.0446],
        [-0.0346, -0.0769, -0.2033,  0.1819, -0.3263,  0.0849, -0.0937,  0.0274,
         -0.0865, -0.1790, -0.5308,  0.0269,  0.2954, -0.1072,  0.2203,  0.0983,
      

In [10]:
# load weight dictionary to see the keys

weights['models']['Feature_Extractor']

OrderedDict([('genomic_feature_extractor.genomic_feature_extractor.0.weight',
              tensor([[-0.4250,  0.2636, -0.4603,  0.0733,  0.4305,  0.0300,  0.2570, -0.4492,
                       -0.1545,  0.1972,  0.1246,  0.0849, -0.1533, -0.1209,  0.2893,  0.5609,
                       -0.3924,  0.1108, -0.2439,  0.0524],
                      [ 0.3815,  0.0620, -0.3089,  0.1661,  0.0566,  0.2817, -0.0772, -0.0423,
                       -0.0132,  0.2168,  0.0512, -0.0400,  0.0677, -0.1994,  0.0775,  0.3982,
                       -0.0377,  0.1775, -0.5042, -0.3299],
                      [-0.5778, -0.1753, -0.0905,  0.1218,  0.1970, -0.3466, -0.4813,  0.1017,
                        0.8698, -0.2419,  0.1321, -0.1542,  0.0931, -0.3106,  0.6785, -0.3515,
                        0.4746, -0.3496,  0.2080, -0.2929],
                      [ 0.5740,  0.0574,  0.3716,  0.1401, -0.2224, -0.3132,  0.1873,  0.6987,
                        0.1025, -0.4229,  0.1138,  0.1346,  0.0536, -0.1835, 